In [94]:
doc = """
This jupyter notebook is authored by ygg_anderson for the Token Engineering Commons. See appropriate licensing. 🐧 🐧 🐧
"""

import param
import panel as pn
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np
from scipy.stats.mstats import gmean
import os
pn.extension()

yellow = '#DEFB48'
blue = '#0F2EEE'
pink = '#FD40FE'
light_blue = '#03B3FF'
purple = '#7622A8'
black = '#0b0a15'
TEC_COLOURS = [blue, black, yellow, pink, purple]
APP_PATH = './'

sheets = [
    "Total Impact Hours so far", 
    "IH Predictions", 
    "#8 Jan 1",
    "#7 Dec 18",
    "#6 Dec 4",
    "#5 Nov 20",
    "#4 Nov 6",
    "#3 Oct 23",
    "#2 Oct 9",
    "#1 Sept 24",
    "#0 Sept 7 (historic)",
] + [f"#{i} IH Results" for i in range(9)]
sheets = {i:sheet for i, sheet in enumerate(sheets)}

def read_excel(sheet_name="Total Impact Hours so far", header=1, index_col=0, usecols=None) -> pd.DataFrame:
    data = pd.read_excel(
        os.path.join(APP_PATH, "data", "TEC Praise Quantification.xlsx"),
        sheet_name=sheet_name,
        engine='openpyxl',
        header=header,
        index_col=index_col,
        usecols=usecols
    ).reset_index().dropna(how='any')
    return data

## Tests
impact_hour_data_1 = read_excel()
impact_hour_data_2 = read_excel(sheet_name="IH Predictions", header=0, index_col=0, usecols='A:I').drop(index=19)
pn.Row(impact_hour_data_1.hvplot.table(), impact_hour_data_2.hvplot.table())

Row
    [0] HoloViews(Table)
    [1] HoloViews(Table)

In [95]:
# Load CSTK data
cstk_data = pd.read_csv('CSTK_DATA.csv', header=None).reset_index().head(100)
cstk_data.columns = ['CSTK Token Holders', 'CSTK Tokens']
cstk_data['CSTK Tokens Capped'] = cstk_data['CSTK Tokens'].apply(lambda x: min(x, cstk_data['CSTK Tokens'].sum()/10))
cstk_data

,CSTK Token Holders,CSTK Tokens,CSTK Tokens Capped
0,0,176256,83367.7
1,1,112192,83367.7
2,2,46263,46263.0
3,3,44539,44539.0
4,4,30783,30783.0
...,...,...,...
95,95,570,570.0
96,96,565,565.0
97,97,563,563.0
98,98,560,560.0


In [155]:
import numpy as np
class ImpactHoursFormula(param.Parameterized):
    """
    Sem's Formula 🌱 🐝 🍯
    This formala was a collaboration of Sem and Griff for the TEC hatch impact hours formula. 
    https://forum.tecommons.org/t/impact-hour-rewards-deep-dive/90/5
    """
    
    # Impact Hour Data
    historic = pd.read_csv('data/IHPredictions.csv').query('Model=="Historic"')
    optimistic =  pd.read_csv('data/IHPredictions.csv').query('Model=="Optimistic"')
    predicted_labour_rate = param.Number(0.5, bounds=(-.5,1.5), step=0.05)

    # Impact Hour Formula
    total_impact_hours = param.Integer(step=100)
    minimum_raise = param.Number(100, bounds=(10, 10000), step=100)
    expected_raise_per_impact_hour = param.Number(25, bounds=(0,200), step=1)
    maximum_impact_hour_rate = param.Number(100, bounds=(0,200), step=1)
    target_raise = param.Number()
    maximum_raise = param.Number()
    
    # Hatch params
    hatch_period_days = param.Integer(15, bounds=(5, 30), step=2)
    hatch_tribute = param.Number(0.05, bounds=(0,1))   
    # CSTK Ratio
    total_cstk_tokens = param.Number(cstk_data['CSTK Tokens Capped'].sum(), constant=True)
    hatch_oracle_ratio = param.Number(0.005, bounds=(0.005, 100), step=0.005)
    # Number of TESTTEC exchanged for 1 wxdai
    hatch_exchange_rate = param.Number(10000, bounds=(1,100000), step=1)  
    
    def __init__(self, **params):
        super(ImpactHoursFormula, self).__init__(**params)
        
        # Initial Predicted Impact Hours
        historic = self.historic.set_index('Round')
        optimistic = self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"].set_index('Round')
        predicted = optimistic.copy()
        predicted['Total IH'] = self.predicted_labour_rate * historic[historic["Actual / Predicted"] == "Predicted"]['Total IH'] + (1 - self.predicted_labour_rate) * optimistic['Total IH']
        predicted['Total Hours'] = self.predicted_labour_rate * historic[historic["Actual / Predicted"] == "Predicted"]['Total Hours'] + (1 - self.predicted_labour_rate) * optimistic['Total Hours']
        self.total_impact_hours = int(predicted['Total IH'].max()) 
        
        # Maximum Raise
        self.maximum_raise = self.total_impact_hours * self.expected_raise_per_impact_hour
        self.param['maximum_raise'].bounds =  (self.maximum_raise / 10, self.maximum_raise * 10)
        self.param['maximum_raise'].step = self.maximum_raise / 10
        
        # Target Raise
        self.target_raise = self.maximum_raise / 2
        self.param['target_raise'].bounds =  (self.minimum_raise, self.maximum_raise)
        self.param['target_raise'].step = self.maximum_raise / 10
        
    def impact_hours_accumulation(self):
        x = 'End Date'

        historic = self.historic.set_index('Round')
        optimistic = self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"].set_index('Round')
        predicted = optimistic.copy()
        predicted['Total IH'] = self.predicted_labour_rate * historic[historic["Actual / Predicted"] == "Predicted"]['Total IH'] + (1 - self.predicted_labour_rate) * optimistic['Total IH']
        predicted['Total Hours'] = self.predicted_labour_rate * historic[historic["Actual / Predicted"] == "Predicted"]['Total Hours'] + (1 - self.predicted_labour_rate) * optimistic['Total Hours']

        historic_curve = historic.hvplot(x, 'Total IH', rot=45, title='Impact Hours Accumulation Curve 🛠️')
        historic_bar = historic.hvplot.bar(x, 'Total Hours', label='Historic')

        optimistic_curve = optimistic.hvplot(x, 'Total IH')
        optimistic_bar = optimistic.hvplot.bar(x, 'Total Hours', label='Optimistic')

        predicted_curve = predicted.hvplot(x, 'Total IH', rot=45, title='Impact Hours Accumulation Curve :)')
        predicted_bar = predicted.hvplot.bar(x, 'Total Hours', label='Predicted')
        
        self.total_impact_hours = int(predicted['Total IH'].max()) 
        return pn.Column(historic_curve * historic_bar * predicted_curve * predicted_bar * optimistic_curve * optimistic_bar) 

    def impact_hours_rewards(self):
        expected_raise = self.total_impact_hours * self.expected_raise_per_impact_hour
        if expected_raise > self.maximum_raise:
            expected_raise = self.maximum_raise
        self.param['maximum_raise'].bounds =  (expected_raise, expected_raise * 10)
        self.param['maximum_raise'].step = expected_raise / 10
        if self.target_raise > self.maximum_raise:
            self.target_raise = self.maximum_raise
        self.param['target_raise'].bounds =  (self.minimum_raise, self.maximum_raise)
        self.param['target_raise'].step = self.maximum_raise / 100
        
        x = np.linspace(self.minimum_raise, self.maximum_raise)

        R = self.maximum_impact_hour_rate

        m = self.expected_raise_per_impact_hour
        
        H = self.total_impact_hours

        y = [R* (x / (x + m*H)) for x in x]

        df = pd.DataFrame([x,y]).T
        df.columns = ['Total XDAI Raised','Impact Hour Rate']
        
       
        try:
            expected_impact_hour_rate = df[df['Total XDAI Raised'] > expected_raise].iloc[0]['Impact Hour Rate']
        except:
            expected_impact_hour_rate = df['Impact Hour Rate'].max()
        try:
            target_impact_hour_rate = df[df['Total XDAI Raised'] > self.target_raise].iloc[0]['Impact Hour Rate']
        except:
            target_impact_hour_rate = df['Impact Hour Rate'].max()
        impact_hours_plot = df.hvplot.area(title='Total Raise and Impact Hour Rate 🎯', x='Total XDAI Raised',  xformatter='%.0f', hover=True)
        
        height = impact_hours_plot.data["Impact Hour Rate"].max() - impact_hours_plot.data["Impact Hour Rate"].min()
        expected = hv.Spikes(([expected_raise], [height]), vdims="height", label="Expected Raise").opts(color='blue', line_width=2) * hv.HLine(expected_impact_hour_rate).opts(color='blue', line_width=2)
        target = hv.Spikes(([self.target_raise], [height]), vdims="height", label="Target Raise").opts(color='red', line_width=2) * hv.HLine(target_impact_hour_rate).opts(color='red', line_width=2)
        
        return (impact_hours_plot * target * expected).opts(legend_position='bottom_right')

    def funding_pools(self):
        x = np.linspace(self.minimum_raise, self.maximum_raise)

        R = self.maximum_impact_hour_rate

        m = self.expected_raise_per_impact_hour
        
        H = self.total_impact_hours

        y = [R* (x / (x + m*H)) for x in x]

        df = pd.DataFrame([x,y]).T
        df.columns = ['Total XDAI Raised','Impact Hour Rate']
        
        # Minimum Results
        minimum_raise = self.minimum_raise
        minimum_rate = df[df['Total XDAI Raised'] > minimum_raise].iloc[0]['Impact Hour Rate']
        minimum_cultural_tribute = self.total_impact_hours * minimum_rate
        
        # Expected Results
        expected_raise = self.total_impact_hours * self.expected_raise_per_impact_hour
        try:
            expected_rate = df[df['Total XDAI Raised'] > expected_raise].iloc[0]['Impact Hour Rate']
        except:
            expected_rate = df['Impact Hour Rate'].max()
            
        expected_cultural_tribute = self.total_impact_hours * expected_rate

        # Target Results
        target_raise = self.target_raise
        try:
            target_rate = df[df['Total XDAI Raised'] > target_raise].iloc[0]['Impact Hour Rate']
        except:
            target_rate = df['Impact Hour Rate'].max()
            
        target_cultural_tribute = self.total_impact_hours * target_rate

        # Funding Pools and Tribute
        funding = pd.DataFrame.from_dict({
            'Mimimum': [minimum_cultural_tribute, minimum_raise-minimum_cultural_tribute],
            'Expected': [expected_cultural_tribute, expected_raise-expected_cultural_tribute],
            'Target': [target_cultural_tribute, target_raise-target_cultural_tribute]}, orient='index', columns=['Culture Tribute', 'Funding Pool'])
        funding_plot = funding.hvplot.bar(title="Funding Pool Outcomes 🔋", stacked=True, ylim=(0,self.maximum_raise),  yformatter='%.0f').opts(color=hv.Cycle(TEC_COLOURS))

        return funding_plot
    
    def hatch_raise_view(self):
        # Load CSTK data
        cstk_data = pd.read_csv('CSTK_DATA.csv', header=None).reset_index().head(100)
        cstk_data.columns = ['CSTK Token Holders', 'CSTK Tokens']
        cstk_data['CSTK Tokens Capped'] = cstk_data['CSTK Tokens'].apply(lambda x: min(x, cstk_data['CSTK Tokens'].sum()/10))
        cstk_data['Cap raise'] = cstk_data['CSTK Tokens Capped'] * self.hatch_oracle_ratio

        cap_plot = cstk_data.hvplot.area(title="Raise Targets Per Hatcher", x='CSTK Token Holders', y='Cap raise', yformatter='%.0f', label="Cap Raise", ylabel="XDAI Staked")

        cstk_data['max_goal'] = cstk_data['Cap raise'] * self.maximum_raise
        max_plot = cstk_data.hvplot.area(x='CSTK Token Holders', y='max_goal', yformatter='%.0f', label="Max Raise")

        cstk_data['min_goal'] = cstk_data['Cap raise'] * self.minimum_raise
        min_plot = cstk_data.hvplot.area(x='CSTK Token Holders', y='min_goal', yformatter='%.0f', label="Min Raise")

        cstk_data['target_goal'] = cstk_data['Cap raise'] * self.target_raise 
        target_plot = cstk_data.hvplot.line(x='CSTK Token Holders', y='target_goal', yformatter='%.0f', label="Target Raise")
        
        raise_bars = cstk_data.iloc[:,3:].sum().sort_values(ascending=False).hvplot.bar(yformatter='%.0f', title="Total Raise Targets")
        
        stats = pd.DataFrame(cstk_data.iloc[:,3:].sum(), columns=['Total XDAI Raise'])
        stats['GMean XDAI Co-vested Per Hatcher'] = gmean(cstk_data.iloc[:,3:])
        stats['XDAI Hatch Tribute'] = stats['Total XDAI Raise'] * self.hatch_tribute
        stats['Total TECH Tokens'] = stats['Total XDAI Raise'] * self.hatch_exchange_rate
        
        
        return pn.Column(cap_plot * max_plot * min_plot * target_plot, raise_bars, stats.sort_values('Total XDAI Raise',ascending=False).apply(round).reset_index().hvplot.table())

In [156]:
impact_hours_rewards = ImpactHoursFormula()

pn.Row(impact_hours_rewards, pn.Column(impact_hours_rewards.impact_hours_accumulation, impact_hours_rewards.impact_hours_rewards, impact_hours_rewards.funding_pools), impact_hours_rewards.hatch_raise_view)

Row
    [0] Column(margin=5, name='ImpactHoursFormula', width=300)
        [0] StaticText(value='<b>ImpactHoursFormula</b>...)
        [1] FloatSlider(end=1.5, name='Predicted labour rate', start=-0.5, step=0.05, value=0.5, value_throttled=0.5)
        [2] IntInput(name='Total impact hours', step=100, value=7406, value_throttled=7406)
        [3] FloatSlider(end=10000, name='Minimum raise', start=10, step=100, value=100, value_throttled=100)
        [4] FloatSlider(end=200, name='Expected raise p..., step=1, value=25, value_throttled=25)
        [5] FloatSlider(end=200, name='Maximum impact h..., step=1, value=100, value_throttled=100)
        [6] FloatSlider(end=185150, name='Target raise', start=100, step=1851.5, value=92575.0, value_throttled=92575.0)
        [7] FloatSlider(end=1851500, name='Maximum raise', start=185150, step=18515.0, value=185150, value_throttled=185150)
        [8] IntSlider(end=30, name='Hatch period days', start=5, step=2, value=15, value_throttled=15)
        [9] FloatSlider(name='Hatch tribute', value=0.05, value_throttled=0.05)
        [10] FloatInput(disabled=True, name='Total cstk tokens', value=711964.4, value_throttled=711964.4)
        [11] FloatSlider(end=100, name='Hatch oracle ratio', start=0.005, step=0.005, value=0.005, value_throttled=0.005)
        [12] FloatSlider(end=100000, name='Hatch exchange rate', start=1, step=1, value=10000, value_throttled=10000)
    [1] Column
        [0] ParamMethod(method)
        [1] ParamMethod(method)
        [2] ParamMethod(method)
    [2] ParamMethod(method)